# SSMEO Lab 01 : Image coordinates, distorsion model

***

### Import packages

In [22]:
import numpy as np
import cv2
from scipy.optimize import fsolve

## Part 1 : Image measurements
***

<div class="alert alert-block alert-info">
<b>Objectives:</b> In this section you will familiarize yourlsef with image format, image coordinates and manual measurements on images.
</div>

### 1.0 Read image, define parameters

Set up the path to your image and path where to save measurements later on

In [8]:
im_path = '/media/topo/Data/ALS/Vallet/Images/18704.jpg'
img = cv2.imread(im_path)

out_path = '/media/topo/Data/ALS/Vallet/Images/'

measurements = {'x':[], 'y':[]}

### 1.1 Perform manual measurements on the image:

Use the provided functions to populate **measurements** dictionnary with manual measurements

**click_event** function is handling user interaction with the displayed image and details how to perform such measurements

In [9]:
def click_event(event, x, y, flags, measurements):
    '''
    On user's double left click : add  measurements (x, y in pixels) to measurements dictionnary
    and draw it on the image
    '''
    buf = 25
    #On double left click :
    if event == cv2.EVENT_LBUTTONDBLCLK:
        #Draw cross at clicked pixel
        cv2.line(img, (x,y-buf), (x,y+buf), (50,90,255), 1)
        cv2.line(img, (x-buf,y), (x+buf,y), (50,90,255), 1)
        cv2.putText(img, f"{x}, {y}",(x+5, y-5),cv2.FONT_HERSHEY_SIMPLEX,1,(50,90,255),2)

        #Update image
        cv2.imshow('Manual measurement', img)
        
        #Store clicked x and y coordinate into measurement dictionnary
        measurements['x'].append(x)
        measurements['y'].append(y)

In [10]:
#Diplay loop on the image :

cv2.namedWindow('Manual measurement', cv2.WINDOW_NORMAL)
cv2.setMouseCallback('Manual measurement', click_event, measurements)
cv2.imshow('Manual measurement', img)
cv2.resizeWindow('Manual measurement', 1200,800)

while True:
    k = cv2.waitKey(100)
    if k == 27: # wait for ESC key to exit
        cv2.destroyAllWindows()
        break        
    if cv2.getWindowProperty('Manual measurement',cv2.WND_PROP_VISIBLE) < 1:# wait for window to be closed        
        break        
cv2.destroyAllWindows()

<div class="alert alert-block alert-info">
Pay attention to the coordinates in pixels when you do a measurement. Where is the origin of X and Y axis ? How are the axis oriented ?
</div>

### 1.2 Define pixel pixel coordinate conversion function

(a) Define function **sensor2perspective** to project pixel coordinates from sensor to perspective centered frame

(b) Define function **perspective2sensor** to project pixel coordinates from perspective to sensor centered frame

In [11]:
def sensor2perspective(img, xy):
    '''
    Project from sensor to perspective coordinate system
    
    Parameters
    ----------
    img : cv2.image
        the image from which take a grid
    xy : np.array(Nx2)
        the array of measurements to project
            
    Return
    ------
    proj_xy : np.array(Nx2)
        the reprojected array    
    '''
    #get image size
    height, width, channels = img.shape
    
    #convert coordinates
    proj_xy = np.empty(xy.shape)
    proj_xy[:,0] = xy[:,0] - width/2
    proj_xy[:,1] = height/2 - xy[:,1]
    
    return proj_xy

#Define function to project from sensor to perspective coordinate system
def perspective2sensor(img, xy):
    '''
    Project from perspective to sensor coordinate system
    
    Parameters
    ----------
    img : cv2.image
        the image from which take a grid
    xy : np.array(Nx2)
        the array of measurements to project
            
    Return
    ------
    proj_xy : np.array(Nx2)
        the reprojected array    
    '''
    #get image size
    height, width, channels = img.shape
    
    #convert coordinates
    proj_xy = np.empty(xy.shape)
    proj_xy[:,0] = xy[:,0] + width/2
    proj_xy[:,1] = height/2 - xy[:,1]
    
    return proj_xy

<div class="alert alert-block alert-info">
Up to now, you used a sensor centered coordinate system :
    <ul>
    <li>The origin is the top-left corner of the image</li>
    <li>X is oriented horizontally →</li>
    <li>Y cis oriented downward ↓</li>
    </ul> 
    
In the next exercise, to undistort images, we will use a perspective centered coordinate system :
    <ul>
    <li>The origin is the perspective center of the image (approximated by the image center in this case)</li>
    <li>X is oriented horizontally →</li>
    <li>Y cis oriented upward ↑</li>
    </ul> 
</div>

In [14]:
#Test that you projections are correct :
height, width, channels = img.shape

sensor_dummy = np.array([[0, 0], [width/2, height/2]])
perspect_dummy = np.array([[-width/2, height/2], [0, 0]])
assert((sensor_dummy == perspective2sensor(img,perspect_dummy)).all())
assert((perspect_dummy == sensor2perspective(img,sensor_dummy)).all())

### 1.3 Save measurements to txt file

#### (a) Convert your measurements to perspective centered coordinate system

In [20]:
xy = np.array([measurements['x'], measurements['y']]).T
print(f"Measurements in sensor coordinates :\n {xy}")

Measurements in sensor coordinates :
 [[8820 1867]
 [8303 7553]
 [4616 5169]
 [8107 4499]
 [2385 1083]
 [1445 5912]]


In [21]:
xy = sensor2perspective(img,xy)
print(f"Measurements in perspective coordinates :\n {xy}")

Measurements in perspective coordinates :
 [[ 3656.  2013.]
 [ 3139. -3673.]
 [ -548. -1289.]
 [ 2943.  -619.]
 [-2779.  2797.]
 [-3719. -2032.]]


#### (b) Save your measurements for latter use

In [18]:
np.savetxt(out_path + 'coord.txt', xy, fmt='%d')
print(f"Saving {xy.shape[0]} measurements to {out_path}coord.txt :")

Measurements in sensor coordinates :
 [[8820 1867]
 [8303 7553]
 [4616 5169]
 [8107 4499]
 [2385 1083]
 [1445 5912]]
Measurements in perspective coordinates :
 [[ 3656.  2013.]
 [ 3139. -3673.]
 [ -548. -1289.]
 [ 2943.  -619.]
 [-2779.  2797.]
 [-3719. -2032.]]
Saving 6 measurements to /media/topo/Data/ALS/Vallet/Images/coord.txt :


## Part 2 : Modelling camera distorsion
***

<div class="alert alert-block alert-info">
<b>Objectives:</b> Cameras are not perfect and distorsion due to imperfect mounting and lens distorsion must me corrected to accurately map each pixel. 
    
In this section you will implement a simple distorsion model and visualize its effect on the pixels' position
</div>

### 2.0 Load image

#### Load your measurements

In [52]:
im_path = '/media/topo/Data/ALS/Vallet/Images/18704.jpg'
out_path = '/media/topo/Data/ALS/Vallet/Images/'

#Load image 
img = cv2.imread(im_path)

#Load raw measurements :
xy = np.loadtxt(out_path + 'coord.txt', delimiter=' ')
print(f"Loaded {xy.shape[0]} measurements :")
print(xy)

Loaded 6 measurements :
[[ 3656.  2013.]
 [ 3139. -3673.]
 [ -548. -1289.]
 [ 2943.  -619.]
 [-2779.  2797.]
 [-3719. -2032.]]


### 2.1 Define camera distorsion model

We will use the simplified Contrady-Brown distorsion model. 
This model relates distorded image coordinates $x_d, y_d$ (the points you just measured), to the undistorded ones $x, y$ through radial coefs ($k_1, k_2$) and tangential coefs ($p_1, p_2$) :
    



\begin{equation*}
x(1 + k_1\cdot r² + k_2 \cdot r⁴) + p_1(r² + 2\cdot x²) + 2\cdot p_2\cdot x\cdot y - x_d = 0
\tag{1}
\end{equation*}
\begin{equation*}
y(1 + k_1\cdot r² + k_2\cdot r⁴) + p_2(r² + 2\cdot y²) + 2\cdot p_1\cdot x\cdot y - y_d = 0
\tag{2}
\end{equation*}

With $r² = x² + y²$      


<div class="alert alert-block alert-info">
$x, y, x_d, x_y$ are expressed in meters in the model.    
You will need to convert from unit of pixel to meters for your measurements given the physical size of pixels of the camera : 
    
$px_{cam} = 5.2e^{-6} [m]$
</div>

#### (a) Define model

We will use scipy non-linear solver to find for any measurment on the distorded image ($x_d, y_d$) the corresponding undistorded measurement ($x, y$).

Define **undisort** function, which returns the two symbolic equations (1) and (2), given model's parameters, and undistorded coordinate of a measurement

Have a look at scipy.optimize.fsolve function to understand its requirements

In [53]:
def undistort(var,x_d,y_d,k1,k2,p1,p2,px_size):  
    '''
    Define equation (1) and (2) given a complet set of parameter and distorded measurement coordinates
    
    Parameters
    ----------
    var : tupple
        tupple containing x and y variables. This are the two variables scipy solver will solve for
        
    x_d, y_d : floats
        the coordinates of a measurment on the distorded image
        
    k1, k2, p1, p2 : floats
        Contrady-Brown distorsion model coefficient
    px_size : float
        physical size of a pixel on the sensor [m]
        
    Return
    ------
    
    list of size 2 containing both eqation of the Contrady-Brown model
    '''
    #Unpack variables
    x, y = var
    
    #Convert x, y from pixels to meters to match distorsion model unit
    x = x*px_size
    y = y*px_size
    x_d = x_d*px_size
    y_d = y_d*px_size
    
    r2 = x*x + y*y

    return [x*(1 + k1*r2 + k2*r2*r2) + p1*(r2+2*x*x) + 2*p2*x*y - x_d,
            y*(1 + k1*r2 + k2*r2*r2) + p2*(r2+2*y*y) + 2*p1*x*y - y_d]

<div class="alert alert-warning">
Not sure where to pu the conversion from pixel unit to meter. If we keep px all along, coef of the distorsion model
become super small (k2 -> 1e-20). Anyway I am not sure how to relate that with a distorsion model from metashape (whixh uses X/Z and Y/Z for x,y to with this manual implementation.

</div>

#### (b) Define function to generate coordinate grid and to plot distorsion correction results :

We would now like to visualize on the image the distorsion model given the distorsion coefficients. 

To do so you can define a regular grid of pixels on the image and evaluate the distorsion model at each positions. You can the use the function **plot_distorsions** to visualize and optionnaly amplify the distorsion vectors

In [54]:
def grid_points(img, k):
    '''
    Generates a regular grid of points from an image (perspective centered frame)
    
    Parameters
    ----------
    img : cv2.image
        the image from which take a grid
    k : int
        the grid spacing in pixel
            
    Return
    ------
    grid : Nx2 np.array
        the array of the grid, each line contains x,y coordinate of a given node
    '''
    height, width, channels = img.shape
    
    x, y = np.meshgrid(np.arange(-width//2,width//2, k),
                       np.arange(-height//2,height//2, k))
    return np.array([x.flatten(), y.flatten()]).T

In [55]:
def plot_distorsions(img, raw_pts, undist_pts, color=(0,0,255), thickness=3):
    '''
    Plots distortion vector on an imga given raw points and the undistorded equivalents
    
    Parameters
    ----------
    img : cv2.image
        the image to plot
        
    raw_pts : np.array (Nx2)
        the original grid without distortion correction
        
    undist_pts : np.array(Nx2)
        the grid with distortion correction

    '''
    for i in range(len(raw_pts)):
        cv2.line(img,
                 (raw_pts[i]).astype(int),
                 (undist_pts[i]).astype(int),
                 color, thickness)
        cv2.circle(img,
                  (raw_pts[i].astype(int))
                   , 2*thickness, color, -1)
    
    cv2.namedWindow('Distorsion Plot', cv2.WINDOW_NORMAL)
    cv2.imshow('Distorsion Plot', img)
    cv2.resizeWindow('Distorsion Plot', 1200,800)

    while True:
        k = cv2.waitKey(100)
        if k == 27: # wait for ESC key to exit
            cv2.destroyAllWindows()
            break        
        if cv2.getWindowProperty('Distorsion Plot',cv2.WND_PROP_VISIBLE) < 1:# wait for window to be closed        
            break        
    cv2.destroyAllWindows()

#### (c) Generate the grid, estimate undistorded coordinates from the model 

In [56]:
#Generate the grid with one point and project into perspective centered coordinates
grid = grid_points(img,100)

In [57]:
#Define distorsion coefficients : 
# Radial coefficients
k1 = 1
k2 = 1
# Tangential coefficients
p1 = 1
p2 = 1
# pixel size
px_size = 5.2e-6

In [58]:
#Apply distorsion model to the grid
undistorded_grid = []
for var in grid:
    undistorded_grid.append(fsolve(undistort,
                                   var,
                                   args=(var[0], var[1],k1,k2,p1,p2,px_size)))
    
undistorded_grid = np.asarray(undistorded_grid)

#### (d) Visualize the result of the undistorded coordinates estimation

In [59]:
#Project both grids to sensor centered coordinates and visualize the results
grid_p = perspective2sensor(img,grid)
undistorded_grid_p = perspective2sensor(img,undistorded_grid)

#Load image & plot
img = cv2.imread(im_path)
plot_distorsions(img, grid_p, undistorded_grid_p)

## Part 3 : Apply camera distorsion model to manual measurements
***

#### (a) Load measurements and estimate undistorded coordinates

In [60]:
undist_xy = []
for var in xy:
    undist_xy.append(fsolve(undistort,
                                   var,
                                   args=(var[0], var[1],k1,k2,p1,p2,px_size)))
undist_xy = np.asarray(undist_xy)   

In [61]:
#Define distorsion coefficients : 
# Radial coefficients
k1 = 1
k2 = 1
# Tangential coefficients
p1 = 1
p2 = 1
# pixel size
px_size = 5.2e-6

#### (b) Project distorded and undistorded measurements to sensor centered frame and plot results

In [63]:
xy_p = perspective2sensor(img, xy)
undist_xy_p = perspective2sensor(img, undist_xy)

#Load image 
img = cv2.imread(im_path)

plot_distorsions(img, xy_p, undist_xy_p)

In [65]:
np.savetxt(out_path + 'coord_undistord.txt', undist_xy_p, fmt='%d')